<a href="https://colab.research.google.com/github/danimonz/Senior-Thesis/blob/main/YoutubeAPI_AWS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import libraries
import requests
import pandas as pd
import time
import psycopg2 as ps

In [2]:
#Keys
API_KEY = #removed for privacy purposes
CHANNEL_ID ="UC3gNmTGu-TTbFPpfSs5kNkg"

SyntaxError: ignored

In [ ]:
def get_video_stats(video_id):
  #collecting view, like, dislike, and comment counts
  url_video_stats = "https://www.googleapis.com/youtube/v3/videos?part=statistics&key="+API_KEY+"&id="+video_id
  response_video_stats = requests.get(url_video_stats).json()

  view_count = response_video_stats['items'][0]['statistics']['viewCount']
  like_count = response_video_stats['items'][0]['statistics']['likeCount']
  favorite_count = response_video_stats['items'][0]['statistics']['favoriteCount']
  if response_video_stats['items'][0]['statistics'].get('dislikeCount') is not None:
    dislike_count = response_video_stats['items'][0]['statistics']['dislikeCount']
  else:
    dislike_count = 0
  if response_video_stats['items'][0]['statistics'].get('commentCount') is not None:
    comment_count = response_video_stats['items'][0]['statistics']['commentCount']
  else:
    comment_count = 0
  
  return view_count, like_count, favorite_count, dislike_count, comment_count


In [ ]:
def get_videos(df, coords):
  """
  Parameters
  df : database
  coords : coordinates of area you are searching in

  """
  #Make API Call for search
  url_video_search = "https://www.googleapis.com/youtube/v3/search?key="+API_KEY+"&part=snippet&location="+coords+"&locationRadius=50mi&maxResults=10&order=viewCount&type=video"

  response_video_search = requests.get(url_video_search).json()
  time.sleep(3) #gives time for the api to collect data before jumping into for loop

  #for loop to extract video_id, video_title, and upload_date from every video in the search response
  for video in response_video_search['items']:
    if video['id']['kind'] == "youtube#video":
      video_id = video['id']['videoId']
      video_title = video['snippet']['title']
      upload_date = video['snippet']['publishedAt']
      upload_date = str(upload_date).split('T')[0]
      #video_id = response['items'][0]['id']['videoId']

      view_count, like_count, favorite_count, dislike_count, comment_count = get_video_stats(video_id)

      #save data in panda df
      #pd.concat([df, pd.DataFrame.from_records([{ 'a': 1, 'b': 2 }])])
      #df = pd.DataFrame.from_records(your_list)
      df = pd.concat([df, pd.DataFrame.from_records({ 'video_id': [video_id], 'video_title': [video_title],
                        'upload_date': [upload_date], 'view_count': [view_count],
                      'like_count': [like_count], 'favorite_count': [favorite_count],
                      'dislike_count': [dislike_count], 'comment_count': [comment_count]})], ignore_index=True)
  return df


In [ ]:
#coordinates for cities
dallas_coord = "32.779167%2C-96.808891"

In [ ]:
#build dataframe
df = pd.DataFrame(columns=["video_id", "video_title", "upload_date", "view_count", "like_count", "favorite_count", "dislike_count", "comment_count"])
df = get_videos(df, dallas_coord)

df

In [ ]:
#Connect to AWS Database
def connect_to_db(host_name, dbname, port, username, password):
  try:
    conn = ps.connect(host=host_name, database=dbname, user=username, password=password, port=port)
  except ps.OperationalError as e:
    raise e
  else:
    print("Connected to Database")

  return conn

In [ ]:
#Create Table

def create_table(curr):
  create_table_command = ("""CREATE TABLE IF NOT EXISTS videos (
                    video_id VARCHAR(255) PRIMARY KEY,
                    video_title TEXT NOT NULL,
                    upload_date DATE NOT NULL DEFAULT CURRENT_DATE,
                    view_count INTEGER NOT NULL,
                    like_count INTEGER NOT NULL,
                    favorite_count INTEGER NOT NULL,
                    dislike_count INTEGER NOT NULL,
                    comment_count INTEGER NOT NULL
                    )""")
  curr.execute(create_table_command)

In [ ]:
def check_if_video_exists(curr, video_id):
  query = ("""SELECT video_id FROM VIDEOS WHERE video_id = %s""")
  
  curr.execute(query, (video_id,))
  return curr.fetchone() is not None

In [ ]:
#update row if video exists, updating stats

def update_row(curr, video_id, video_title, view_count, like_count, favorite_count, dislike_count, comment_count):
  query = ("""UPDATE videos
           SET video_title = %s,
            view_count = %s,
            like_count = %s,
            favorite_count = %s,
            dislike_count = %s,
            comment_count = %s
           WHERE video_id = %s;""")
  vars_to_update = (video_title, view_count, like_count, favorite_count, dislike_count, comment_count)
  curr.execute(query, vars_to_update)

In [ ]:
#insert command, addin data to aws database
def insert_into_table(curr, video_id, video_title, upload_date, view_count, like_count, favorite_count, dislike_count, comment_count):
    insert_into_videos = ("""INSERT INTO videos (video_id, video_title, upload_date,
                        view_count, like_count, favorite_count, dislike_count, comment_count)
    VALUES(%s,%s,%s,%s,%s,%s,%s,%s);""")
    row_to_insert = (video_id, video_title, upload_date, view_count, like_count, favorite_count, dislike_count, comment_count)
    curr.execute(insert_into_videos, row_to_insert)


In [ ]:
def append_from_df_to_db(curr, df):
  for i, row in df.iterrows():
    insert_into_table(curr, row['video_id'], row['video_title'], row['upload_date'], row['view_count'], row['like_count'], row['favorite_count'], row['dislike_count'], row['comment_count'])

In [ ]:
#Updating the database

#checking if vidoe exists

def update_db(curr, df):
  #temporary dataframe
  tmp_df = pd.DataFrame(columns=["video_id", "video_title", "upload_date", "view_count", "like_count", 
                                  "favorite_count", "dislike_count", "comment_count"])
  
  for i, row in df.iterrows():
    if check_if_video_exists(curr, row['video_id']): #if video exists, update stats
      update_row(curr, row['video_id'], row['video_title'], row['view_count'], row['like_count'], row['favorite_count'], row['dislike_count'], row['comment_count'])
    else: #if video doesn't exist, update data
      tmp_df = pd.concat([tmp_df, pd.DataFrame.from_records({ 'video_id': [row['video_id']], 'video_title': [row['video_title']],
                        'upload_date': [row['upload_date']], 'view_count': [row['view_count']],
                      'like_count': [row['like_count']], 'favorite_count': [row['favorite_count']],
                      'dislike_count': [row['dislike_count']], 'comment_count': [row['comment_count']]})], ignore_index=True)
  
  return tmp_df


In [ ]:
#Database info
host_name = #removed for privacy purposes
dbname = "postgres"
port = #removed for privacy purposes
username = #removed for privacy purposes
password = #removed for privacy purposes
conn = None

conn = connect_to_db(host_name, dbname, port, username, password)

In [ ]:
curr = conn.cursor()

In [ ]:
create_table(curr)

In [ ]:
new_vid_df = update_db(curr, df)

In [ ]:
append_from_df_to_db(curr, new_vid_df)

In [ ]:
conn.commit()